# Data Processing Pipeline

This notebook prepares agricultural yield and climate data for modeling.

**Goals:**
- Clean and harmonize yield data
- Process climate variables
- Aggregate results at Year × Municipality × Crop level

---

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

OUTDIR = Path("./prepared")
OUTDIR.mkdir(parents=True, exist_ok=True)

## 1. Load raw datasets

In [20]:
df_yields = pd.read_csv("./data/Yields.csv")
df_yields.columns = df_yields.columns.str.strip()
print(df_yields.shape)

# Basic sanity checks
n_crops = df_yields["Crop"].nunique(dropna=True)
n_munis = df_yields["Municipality"].nunique(dropna=True)
print(f"- number of different crops: {n_crops}")
print(f"- number of different municipalities: {n_munis}")

df_yields.head()


(44448, 9)
- number of different crops: 53
- number of different municipalities: 99


,Year,Municipality,Crop,Variety,Farms,Acres,Yield/Acre,t,i
0,1996,ALEXANDER,ARGENTINE CANOLA,INNOVATOR (HCN 92) (LT),3,575.0,0.579,1,1
1,1996,ALEXANDER,ARGENTINE CANOLA,QUANTUM (91-21864 NA),6,1125.0,0.780,1,1
2,1996,ALEXANDER,ARGENTINE CANOLA,45A71 (NS1471)(ST),4,1012.0,0.756,1,1
3,1996,ALEXANDER,BARLEY,ROBUST,9,1247.0,1.607,1,1
4,1996,ALEXANDER,CANARYSEED,KEET,3,535.0,0.184,1,1


## 1. Load raw datasets

In [21]:
df_weather = pd.read_csv("./data/Weather Reanalysis.csv")
df_weather.columns = df_weather.columns.str.strip()
print(df_weather.shape)

n_munis = df_weather["Municipality"].nunique(dropna=True)
print(f"- number of different municipalities: {n_munis}")
df_weather.head()


(490896, 9)
- number of different municipalities: 84


,Municipality,Date/Time,Year,Month,Day,Max_Temp,Min_Temp,Mean_Temp,Ptol
0,ALEXANDER,1/1/96,1996,1,1,-10.416400,-21.008989,-15.712695,-1.370000e-10
1,ALEXANDER,1/2/96,1996,1,2,-15.238113,-18.910885,-17.074499,-1.370000e-10
2,ALEXANDER,1/3/96,1996,1,3,-13.913316,-21.768749,-17.841032,1.088851e-03
3,ALEXANDER,1/4/96,1996,1,4,-22.517648,-28.702039,-25.609844,-1.370000e-10
4,ALEXANDER,1/5/96,1996,1,5,-23.918762,-30.488448,-27.203605,-1.370000e-10


## 4. Aggregation and feature engineering

In [22]:
YEARS = range(1996, 2012)  # 1996–2011

def prepare_yields_year_muni_crop(df):
    """Aggregate Yields.csv to one row per (Year, Municipality, Crop).

    Mean_Yield is the mean yield across varieties for the same (Year, Municipality, Crop).
    Recommended definition: area-weighted mean of Yield/Acre using Acres:
        Mean_Yield = sum(Yield/Acre * Acres) / sum(Acres)
    If Acres is missing/zero for a group, fall back to simple mean of Yield/Acre.
    """
    y = df.copy()
    y.columns = y.columns.str.strip()

    # Required columns check
    required = {"Year", "Municipality", "Crop", "Yield/Acre"}
    missing = required - set(y.columns)
    if missing:
        raise ValueError(f"Missing required columns in Yields.csv: {sorted(missing)}")

    # Types
    y["Year"] = pd.to_numeric(y["Year"], errors="coerce")
    y["Yield/Acre"] = pd.to_numeric(y["Yield/Acre"], errors="coerce")

    if "Acres" in y.columns:
        y["Acres"] = pd.to_numeric(y["Acres"], errors="coerce")
    else:
        y["Acres"] = np.nan  # will trigger fallback to simple mean

    # Clean strings
    y["Municipality"] = y["Municipality"].astype(str).str.strip().str.upper()
    y["Crop"] = y["Crop"].astype(str).str.strip()

    # Filter study window and remove invalid rows
    y = y[y["Year"].between(min(YEARS), max(YEARS))].copy()
    y = y.dropna(subset=["Year", "Municipality", "Crop", "Yield/Acre"])

    # Weighted mean yield per acre (weights = Acres)
    y["_acres"] = y["Acres"].fillna(0)
    y["_prod"] = y["Yield/Acre"] * y["_acres"]

    grouped = (
        y.groupby(["Year", "Municipality", "Crop"], as_index=False)
         .agg(
             acres_sum=("_acres", "sum"),
             prod_sum=("_prod", "sum"),
             yield_mean_simple=("Yield/Acre", "mean"),
         )
    )

    grouped["Mean_Yield"] = np.where(
        grouped["acres_sum"] > 0,
        grouped["prod_sum"] / grouped["acres_sum"],
        grouped["yield_mean_simple"]
    )

    out = grouped[["Year", "Municipality", "Crop", "Mean_Yield"]].copy()
    out["Year"] = out["Year"].astype(int)
    return out


## 5. Export prepared dataset

In [23]:
yields_by_crop = prepare_yields_year_muni_crop(df_yields)

print("Prepared yields_by_crop:", yields_by_crop.shape)
print("Unique key check (Year, Municipality, Crop):", yields_by_crop.duplicated(["Year","Municipality","Crop"]).sum())

yields_by_crop.to_csv(OUTDIR / "yields_by_crop.csv", index=False)
yields_by_crop.head()


Prepared yields_by_crop: (14476, 4)
Unique key check (Year, Municipality, Crop): 0


,Year,Municipality,Crop,Mean_Yield
0,1996,ALEXANDER,ARGENTINE CANOLA,0.728428
1,1996,ALEXANDER,BARLEY,1.607000
2,1996,ALEXANDER,CANARYSEED,0.184000
3,1996,ALEXANDER,OATS,1.215000
4,1996,ALEXANDER,POLISH CANOLA,0.278000


## 3. Weather indices construction

In [24]:
GROW_START, GROW_END = ("05-01", "10-31")  # May–Oct (optional)

def parse_date_us_2digit_year(s):
    return pd.to_datetime(s, format="%m/%d/%y", errors="coerce")

def weather_indices_biweekly(
    weather_df: pd.DataFrame,
    *,
    municipality_col: str = "Municipality",
    date_col: str = "Date/Time",            # daily date column (e.g. "1/1/96")
    year_col_out: str = "Year",             # output key name
    max_temp_col: str = "Max_Temp",
    min_temp_col: str = "Min_Temp",
    mean_temp_col: str = "Mean_Temp",
    precip_total_col: str | None = "Ptol",  # daily total precip
    rain_col: str | None = None,
    snow_col: str | None = None,
    base_temp_c: float = 18.0,              # for HDD/CDD
    clip_precip_at_zero: bool = True,
    restrict_to_growing_season: bool = True # set False if you want 01..12
) -> pd.DataFrame:
    """
    Output: one row per (Year, Municipality) with columns like:
      05_Q1_Mean_Temp, 05_Q1_CDD, 05_Q1_Ptol, ...
    where:
      MM = month (01..12), Q1 = days 1-15, Q2 = days 16-end.
    """

    df = weather_df.copy()
    df.columns = df.columns.str.strip()
    df[municipality_col] = df[municipality_col].astype(str).str.strip().str.upper()

    # --- parse date and derive time parts (matches the "prefix" logic in your sheet) ---
    df["__date"] = parse_date_us_2digit_year(df[date_col])
    df["__year"] = df["__date"].dt.year
    df["__month"] = df["__date"].dt.month
    df["__day"] = df["__date"].dt.day
    df["Half"] = np.where(df["__day"] <= 15, "Q1", "Q2")

    # Optional: restrict to growing season (May–Oct)
    if restrict_to_growing_season:
        mmdd = df["__date"].dt.strftime("%m-%d")
        season = (mmdd >= GROW_START) & (mmdd <= GROW_END)
        df = df.loc[season].copy()

    # --- numeric coercions ---
    for c in [max_temp_col, min_temp_col, mean_temp_col, precip_total_col, rain_col, snow_col]:
        if c is not None and c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # If Mean_Temp missing but Max/Min exist, compute it (keeps “logic” consistent with meaning)
    if mean_temp_col not in df.columns and (max_temp_col in df.columns and min_temp_col in df.columns):
        df[mean_temp_col] = 0.5 * (df[max_temp_col] + df[min_temp_col])

    # --- HDD / CDD from daily mean temp (as in your current logic) ---
    tmean = df[mean_temp_col]
    df["_HDD"] = np.maximum(0.0, base_temp_c - tmean)
    df["_CDD"] = np.maximum(0.0, tmean - base_temp_c)

    # --- precipitation (match sheet variables: Ptol, Total Rain, Total Snow) ---
    if precip_total_col is not None and precip_total_col in df.columns:
        if clip_precip_at_zero:
            df[precip_total_col] = df[precip_total_col].clip(lower=0)

    if rain_col is not None and rain_col in df.columns:
        if clip_precip_at_zero:
            df[rain_col] = df[rain_col].clip(lower=0)

    if snow_col is not None and snow_col in df.columns:
        if clip_precip_at_zero:
            df[snow_col] = df[snow_col].clip(lower=0)

    # If Rain/Snow not available, fall back to Ptol + 0 snow (same as your current behavior)
    df["_TotalRain"] = df[rain_col] if (rain_col is not None and rain_col in df.columns) else (
        df[precip_total_col] if (precip_total_col is not None and precip_total_col in df.columns) else np.nan
    )
    df["_TotalSnow"] = df[snow_col] if (snow_col is not None and snow_col in df.columns) else 0.0
    df["_Ptol"] = df[precip_total_col] if (precip_total_col is not None and precip_total_col in df.columns) else (
        df["_TotalRain"] + df["_TotalSnow"]
    )

    # --- aggregate daily -> (Year, Municipality, Month, Half) ---
    grp_cols = ["__year", municipality_col, "__month", "Half"]
    agg = {
        "_HDD": "sum",
        "_CDD": "sum",
        "_TotalRain": "sum",
        "_TotalSnow": "sum",
        "_Ptol": "sum",
        max_temp_col: "mean" if max_temp_col in df.columns else "mean",
        min_temp_col: "mean" if min_temp_col in df.columns else "mean",
        mean_temp_col: "mean",
    }
    # Only keep keys that exist
    agg = {k: v for k, v in agg.items() if k in df.columns}

    bi = df.groupby(grp_cols, dropna=False).agg(agg).reset_index()

    # rename to exactly match your sheet abbreviations
    rename_map = {
        "__year": year_col_out,
        "__month": "Month2",
        "_CDD": "CDD",
        "_HDD": "HDD",
        "_Ptol": "Ptol",
        "_TotalRain": "Total Rain",
        "_TotalSnow": "Total Snow",
        max_temp_col: "Max_Temp" if max_temp_col in bi.columns else max_temp_col,
        min_temp_col: "Min_Temp" if min_temp_col in bi.columns else min_temp_col,
        mean_temp_col: "Mean_Temp",
    }
    bi = bi.rename(columns={k: v for k, v in rename_map.items() if k in bi.columns})

    # --- wide pivot: (Year, Municipality) x [MM_Q1/MM_Q2 metrics] ---
    bi["MonthHalf"] = bi["Month2"].apply(lambda m: f"{int(m):02d}") + "_" + bi["Half"]
    id_cols = [year_col_out, municipality_col]
    value_cols = [c for c in bi.columns if c not in id_cols + ["Month2", "Half", "MonthHalf"]]

    wide = bi.pivot_table(index=id_cols, columns="MonthHalf", values=value_cols, aggfunc="first")
    wide.columns = [f"{mh}_{metric}" for metric, mh in wide.columns]  # MM_Qx_<Metric>
    wide = wide.reset_index()

    return wide


## 5. Export prepared dataset

In [25]:
weather_idx_raw = weather_indices_biweekly(df_weather)
weather_idx_raw.to_csv(OUTDIR / "weather_indices_raw.csv", index=False)
print("Prepared weather indices:", weather_idx_raw.shape)
weather_idx_raw.head()


Prepared weather indices: (1344, 98)


,Year,Municipality,05_Q1_CDD,05_Q2_CDD,06_Q1_CDD,06_Q2_CDD,07_Q1_CDD,07_Q2_CDD,08_Q1_CDD,08_Q2_CDD,...,06_Q1_Total Snow,06_Q2_Total Snow,07_Q1_Total Snow,07_Q2_Total Snow,08_Q1_Total Snow,08_Q2_Total Snow,09_Q1_Total Snow,09_Q2_Total Snow,10_Q1_Total Snow,10_Q2_Total Snow
0,1996,ALEXANDER,0.0,1.406025,32.701730,41.166500,31.423716,15.575997,36.851627,46.023453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1996,ALONSA,0.0,2.014729,37.197368,35.920028,39.128350,18.739317,31.135125,51.413770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,ARGYLE,0.0,0.152813,24.440870,34.404783,27.131971,17.745561,28.276071,43.549495,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1996,ARMSTRONG,0.0,2.287541,35.589788,36.770040,36.366397,17.175362,32.882947,48.528054,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1996,BIFROST-RIVERTON,0.0,2.287541,35.589788,36.770040,36.366397,17.175362,32.882947,48.528054,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Merge yield and weather data

In [26]:
yields_weather_by_crop = yields_by_crop.merge(
    weather_idx_raw,
    on=["Municipality", "Year"],
    how="inner"
)

print("Merged yields + weather:", yields_weather_by_crop.shape)

# Report missing climate coverage if using left join (optional diagnostics)
# missing = yields_by_crop.merge(weather_idx_raw, on=["Municipality","Year"], how="left", indicator=True)
# print("Rows without climate:", (missing["_merge"] == "left_only").sum())

yields_weather_by_crop.to_csv(OUTDIR / "yields_weather_by_crop.csv", index=False)
yields_weather_by_crop.head()


Merged yields + weather: (11721, 100)


,Year,Municipality,Crop,Mean_Yield,05_Q1_CDD,05_Q2_CDD,06_Q1_CDD,06_Q2_CDD,07_Q1_CDD,07_Q2_CDD,...,06_Q1_Total Snow,06_Q2_Total Snow,07_Q1_Total Snow,07_Q2_Total Snow,08_Q1_Total Snow,08_Q2_Total Snow,09_Q1_Total Snow,09_Q2_Total Snow,10_Q1_Total Snow,10_Q2_Total Snow
0,1996,ALEXANDER,ARGENTINE CANOLA,0.728428,0.0,1.406025,32.70173,41.1665,31.423716,15.575997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1996,ALEXANDER,BARLEY,1.607000,0.0,1.406025,32.70173,41.1665,31.423716,15.575997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,ALEXANDER,CANARYSEED,0.184000,0.0,1.406025,32.70173,41.1665,31.423716,15.575997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1996,ALEXANDER,OATS,1.215000,0.0,1.406025,32.70173,41.1665,31.423716,15.575997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1996,ALEXANDER,POLISH CANOLA,0.278000,0.0,1.406025,32.70173,41.1665,31.423716,15.575997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Principal Component Analysis (PCA / ACP) — Motivation

After merging yield data with biweekly weather reanalysis indices, the resulting dataset contains many climate variables.
Several of these indices are correlated (they measure related aspects of temperature/precipitation/energy balance), which makes
direct interpretation difficult and can introduce redundancy in later analyses.

To summarize the climate information, we apply a Principal Component Analysis (PCA) on the **standardized climate indices**.
PCA creates new variables (principal components) that capture the main directions of variability in the climate data.
These components will be used for visualization and can later serve as compact predictors in statistical models.

## 5. Export prepared dataset

In [27]:
# =========================
# PCA / ACP (PROCESSING)
# =========================
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

DF_NAME = "yields_weather_by_crop"  
OUT_DIR = "./prepared"             # <-- where to save outputs
TAG = "allcrops"                   # e.g. "allcrops" or "wheat_only"

# Which columns are identifiers (NOT used in PCA)
ID_COLS = ["Year", "Municipality", "Crop"]  

# Yield column name (kept for later, not in PCA by default)
YIELD_COL = "Mean_Yield"                   

# Whether to include yield in PCA 
INCLUDE_YIELD_IN_PCA = False

# Optional: filter to a single crop for PCA (set to None to keep all)
CROP_FOR_PCA = None  # e.g. "Wheat" or None

# ---- LOAD df from your notebook namespace ----
df = globals()[DF_NAME].copy()

# Optional crop filter
if CROP_FOR_PCA is not None and "Crop" in df.columns:
    df = df[df["Crop"] == CROP_FOR_PCA].copy()
    TAG = f"{TAG}_{str(CROP_FOR_PCA).lower()}"

# ---- Select PCA input columns ----
# Start from numeric columns only
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove IDs / non-features
for c in ID_COLS:
    if c in num_cols:
        num_cols.remove(c)

# Optionally remove yield
if (not INCLUDE_YIELD_IN_PCA) and (YIELD_COL in num_cols):
    num_cols.remove(YIELD_COL)

# If you have other numeric-but-not-feature columns, exclude them here:
# EXCLUDE_NUM = ["some_index_col"]
# num_cols = [c for c in num_cols if c not in EXCLUDE_NUM]

X = df[num_cols].copy()

# ---- Missing values -> impute (median) ----
imputer = SimpleImputer(strategy="median")
X_imp = imputer.fit_transform(X)

# ---- Standardize (center-reduce) ----
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imp)

# ---- PCA fit ----
pca = PCA()
scores = pca.fit_transform(X_scaled)  # coordinates of observations in PC space

# ---- Build outputs ----
# Explained variance
explained = pd.DataFrame({
    "component": [f"PC{i+1}" for i in range(len(pca.explained_variance_ratio_))],
    "explained_variance_ratio": pca.explained_variance_ratio_,
    "explained_variance_ratio_cum": np.cumsum(pca.explained_variance_ratio_)
})

# Loadings: coefficients of variables in each PC (shape: n_features x n_components)
# We'll save components_.T with nice labels
loadings = pd.DataFrame(
    pca.components_.T,
    index=num_cols,
    columns=[f"PC{i+1}" for i in range(pca.components_.shape[0])]
)

# Scores: keep IDs + yield + PC columns
pc_cols = [f"PC{i+1}" for i in range(scores.shape[1])]
scores_df = pd.DataFrame(scores, columns=pc_cols, index=df.index)

keep_cols = [c for c in ID_COLS if c in df.columns]
if YIELD_COL in df.columns:
    keep_cols.append(YIELD_COL)

scores_out = pd.concat([df[keep_cols].reset_index(drop=True), scores_df.reset_index(drop=True)], axis=1)

# ---- Save ----
os.makedirs(OUT_DIR, exist_ok=True)

explained_path = os.path.join(OUT_DIR, f"pca_explained_{TAG}.csv")
loadings_path  = os.path.join(OUT_DIR, f"pca_loadings_{TAG}.csv")
scores_path    = os.path.join(OUT_DIR, f"pca_scores_{TAG}.csv")

explained.to_csv(explained_path, index=False)
loadings.to_csv(loadings_path)
scores_out.to_csv(scores_path, index=False)

print("Saved:")
print(" -", explained_path)
print(" -", loadings_path)
print(" -", scores_path)

print("\nPCA used", len(num_cols), "variables.")
print("Top cumulative variance:")
print(explained.head(5))

Saved:
 - ./prepared\pca_explained_allcrops.csv
 - ./prepared\pca_loadings_allcrops.csv
 - ./prepared\pca_scores_allcrops.csv

PCA used 96 variables.
Top cumulative variance:
  component  explained_variance_ratio  explained_variance_ratio_cum
0       PC1                  0.225778                      0.225778
1       PC2                  0.126898                      0.352676
2       PC3                  0.097227                      0.449903
3       PC4                  0.085986                      0.535889
4       PC5                  0.066469                      0.602359
